In [150]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import itertools
from tqdm import tqdm


In [151]:
def get_nutrients(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    txt = soup.find(class_='partial recipe-nutrition-section').find(class_="section-body").text
    #Remove white spaces
    txt = txt.strip()
    txt = txt.split('\n')[0]
    #split by ;
    txtf = txt.split(';')
    #extract numbers into separate list
    nums = []
    nutrient = []
    for val in txtf:
        v = val.split()
        num = [x for x in v if sum(char.isdigit() for char in x) > 0]
        nut = [x.lower() for x in v if sum(char.isdigit() for char in x) == 0]
        nums += num
        nutrient += nut
    d = dict(zip(nutrient, nums))
    #turn dict into df
    df = pd.DataFrame(data = [d])
    df['url'] = str(URL)
    return df

In [152]:
def get_recipe_urls():
    #get allrecipes slow cooker links
    url = 'https://www.allrecipes.com/recipes/253/everyday-cooking/slow-cooker/'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    #Turn soup output into string and regex find all recipes
    t = str(soup)
    l = re.findall('(?<="url": ")https:\/\/www\.allrecipes\.com\/recipe\/.*(?=")', t)
    l #outputs list of urls with slow cooker recipes
    return l

In [ ]:
#Establish dataframe of link, nutrient info
df = pd.DataFrame(columns = ['calories', 'carbohydrates', 'cholesterol', 'fat', 'protein', 'sodium', 'url'])
#Get recipe URLs
url_list = get_recipe_urls()

In [153]:
#Loop through unique urls and get nutrients and add to df
for u in tqdm(list(set(url_list))):  
    #Get nutrients
    n = get_nutrients(u)
    #Merge nutrient data with df
    df = pd.concat([df, n],ignore_index = True, sort=True)
    df
print(df)

100%|██████████████████████████████████████████████████████████████████████████████████| 84/84 [00:30<00:00,  2.76it/s]

   calories carbohydrates cholesterol    fat protein     sodium  \
0       251         35.7g         NaN    11g    3.6g     200mg.   
1       415         41.1g      86.5mg  14.1g   29.8g  1281.2mg.   
2       403         30.6g      91.2mg  21.7g   22.4g     884mg.   
3       268         30.4g      23.4mg   9.3g   15.6g   966.9mg.   
4       325         52.6g      22.4mg  10.2g    7.4g    73.2mg.   
5       414          5.5g       143mg  29.4g   30.1g   376.4mg.   
6       190         29.8g        89mg   5.6g      5g   197.6mg.   
7        90         15.6g       8.3mg   2.3g    2.2g      42mg.   
8       480         17.2g     102.6mg  33.3g   30.6g     442mg.   
9       501         32.4g       117mg  29.6g   24.3g  1067.4mg.   
10      465         57.3g      70.3mg    13g   29.8g   733.9mg.   
11      140         37.3g         NaN   0.5g    1.1g     0.4mg.   
12      385           37g      44.8mg    18g   18.1g  1244.6mg.   
13      314           10g     171.3mg     8g     48g  1881.7mg

In [154]:
#save df to csv
df.to_csv('recipe_nutrients.csv', index=False)